In [1]:
import os
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import matplotlib.cm as cm

In [2]:
SimPath = '/Users/sahand/Research/NeuroNet/Data/07-31-2017/Sim2_N5120_L1000_S10_D400_T60000'

# Fixing Storage Path in Parameters File 

In [6]:
def FixPathsParallel(datafolder="/Users/sahand/Research/NeuroNet/Data/Sim5_0626201"):
    folders = os.listdir(datafolder)
    visfolder = datafolder+"/"+"Vis"
    if not os.path.exists(visfolder):
        os.makedirs(visfolder)
    with open(datafolder+"/Parameters.json", 'r') as file:
        data = json.load(file)
        data["DataFolder"]=datafolder
    with open(datafolder+"/Parameters.json", 'w') as file:
        json.dump(data, file, indent=4, separators=(',', ': '))

In [8]:
FixPathsParallel(datafolder=SimPath)

# Simulation Summary

In [9]:
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        html = ['<table align="left" width=100%>']
        html.append('<tr>')
        html.append('<th align="left">Paramater Name</th>')
        html.append('<th align="left">Paramater Value</th>')
        html.append("<tr>")
        for key, value in iter(self.items()):
            html.append("<tr>")
            html.append("<td>{0}</td>".format(str(key)))
            html.append("<td>{0}</td>".format(str(value)))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [10]:
with open(SimPath+"/Parameters.json") as data_file:
    metadata = json.load(data_file)

In [11]:
DictTable(metadata)

Paramater Name,Paramater Value
C,20
C_v,1
ConnectionScale,10
DataFolder,/Users/sahand/Research/NeuroNet/Data/07-31-2017/Sim2_N5120_L1000_S10_D400_T60000
I,0
I_v,0.1
JobID,7258744.bw
NetworkDevel,400
NeuronsPerFile,200
NumberOfFiles,26


# Individual Neuron Time Series

In [12]:
def ReadFileIndividualNeuron(filename, column=0):
    filedata = []
    with open(filename, 'r') as f:
        for line in f:
            ld = []
            dataline = line.rstrip(', \n').lstrip(' ').split(',')
            filedata.append(np.array([dataline[0],dataline[column+1]])) # +1 is here becuase the first column is always time. 
    return np.array(filedata)

In [13]:
def VisTimeSeries(datafolder, neurons=[0],output='Save'):
    
    with open(datafolder+"/Parameters.json") as data_file:
            metadata = json.load(data_file)
    p = metadata["NeuronsPerFile"]
    
    filenumbers = []
    datacolumns = []
    for n in neurons:
        filenumbers.append(int(n/p))
        datacolumns.append(n%p)
    
    datafiles = glob.glob(datafolder+"/*.dat")
    
    files = [datafiles[n] for n in filenumbers]

    fig = plt.figure(figsize=(25,2*len(files)))
    
    for i, file in enumerate(files):
        ax = fig.add_subplot(len(files),1,i+1)
        data = ReadFileIndividualNeuron(file,column=datacolumns[i])
        p=plt.plot(data[:,0], data[:,1], rasterized=True)
        plt.ylim([-90,90])
        plt.setp(p,'Color', [0.7,0.3,0.3], 'linewidth', 3)
        plt.ylabel('{}'.format(neurons[i]))
        plt.grid(True)
    plt.xlabel('Time')
    if output=='Show':
        plt.show()    
    else:
        plt.savefig(datafolder +'/Vis/IndividualTimeSeries.png', bbox_inches='tight')

In [14]:
VisTimeSeries(SimPath,neurons=[1,10,500,1000],output='Save')

# Aggregate Time Frequency Plot

In [15]:
def ReadFile(filename, filenumber=0, perfile=200, threashold=40):
    filedata = []
    with open(filename, 'r') as f:
        for line in f:
            ld = []
            dataline = line.rstrip(', \n').lstrip(' ').split(',')
            for i,data in enumerate(dataline[1:]):
                if data == 'none':
                    data = 0
                if float(data) > threashold:
                    ld.append(np.array([float(dataline[0]), perfile*filenumber + int(i)]))
            
            if len(ld)>0:
                filedata.append(ld)
    return filedata

In [16]:
def VisTimeFreq(datafolder, cuttoff=50, output = 'Save',figsize=(25,10)):
    
    with open(datafolder+"/Parameters.json") as data_file:
            metadata = json.load(data_file)
    
    datafiles = glob.glob(datafolder+"/*.dat")

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1,1,1)

    for i,file in enumerate(datafiles):
        f = ReadFile(file,filenumber=i,perfile=metadata["NeuronsPerFile"],threashold=cuttoff)
        if len(f)>0:
            data = np.array([item for sublist in f for item in sublist])
            ax.scatter(data[:,0], data[:,1], c=[0.7]*3, s=2, rasterized=True)
    if output=='Show':
        plt.show()    
    else:
        plt.savefig(datafolder +'/Vis/TimeFreq.png', bbox_inches='tight')

In [17]:
VisTimeFreq(SimPath,70,output='Save',figsize=(25,15))

# Sorting Neurons

In [18]:
def SortNeurons():
    d = []
    neurontoindex = {}
    coordinates = np.genfromtxt(SimPath+"/Positions.csv", delimiter=',')
    for nxy in coordinates:
        d.append((nxy[0],nxy[1]**2+nxy[2]**2))
    sl = sorted(d, key=lambda x: x[1])
    SortedNeurons = [int(x[0]) for x in sl]
    for i,n in enumerate(SortedNeurons):
        neurontoindex[n] = i
    
    return neurontoindex

# Network

In [19]:
def VisAdjacencyMatrix(datafolder, sort=False, numberoffiles=2, output='Save',figsize=(25,25)):
    
    with open(datafolder+"/Parameters.json") as data_file:
            metadata = json.load(data_file)
    
    networkfiles = glob.glob(datafolder+"/Network/*.csv")
    
    if numberoffiles=='All':
        files = networkfiles
    else:
        files = networkfiles[0:numberoffiles]
    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1,1,1)
    
    if sort:
        SortedNeruons = SortNeurons()
    
    for i,file in enumerate(files):
        data = np.genfromtxt(file, delimiter=',')
        if sort:
            xdata = np.array([SortedNeruons[d] for d in data[:,0]])
            ydata = np.array([SortedNeruons[d] for d in data[:,1]])
        else:
            xdata = data[:,0]
            ydata = data[:,1]
        weights = data[:,2]
        plt.scatter(xdata, ydata, c=weights, s=5, cmap='Blues', rasterized=True)
    if sort:
        ax.set_xticklabels(SortedNeruons.keys())
        ax.set_yticklabels(SortedNeruons.keys())
    plt.colorbar()
    if output=='Show':
        plt.show()    
    else:
        plt.savefig(datafolder +'/Vis/AdjacencyMatrix.png', bbox_inches='tight')

In [ ]:
VisAdjacencyMatrix(SimPath, sort=True, numberoffiles='All', output='Save', figsize=(25,20))

# Scratch

In [17]:
def DevelopNetwork(N=800):
    WeightsP = []
    for i in range(N):
        for j in range(N):
            wd = GetWeight(i,j)
            WeightsP.append(np.array([i,j,wd]))

    return np.array(WeightsP)
    
def GetWeight(n1=1, n2=2):
    if n1==n2:
        w = 0
    else:
        d = np.sqrt(Distance2(n1, n2))
        w = min(int(np.random.normal(100,5)*np.exp(-d/(np.random.normal(20,1)))), 500)
    return w

def Distance2(a, b):
        return (a - b)**2

In [18]:
data=DevelopNetwork(800)

In [19]:
fig = plt.figure(figsize=(15,13))
ax = fig.add_subplot(1,1,1)
data = np.array(data)
xdata = data[:,0]
ydata = data[:,1]
ws = data[:,2]
plt.scatter(xdata, ydata, c=ws, s=40, cmap='Blues', rasterized=True)
plt.colorbar()
#plt.show()